# Import Packages

In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW, Adam
from base import get_basic_model, get_model_passt
from models.mobileViTCA import make_model
from dataset import train_set, test_set, validation_set
from vanilla_kd import VanillaKD

# Loading Models

## Teacher Model

In [12]:
teacher = get_basic_model(mode="logits")

teacher.net = get_model_passt(arch="passt_s_swa_p16_128_ap476",  n_classes=50)
# Loading weights from ESC50 pretrained model
state_dict = torch.hub.load_state_dict_from_url(url='https://github.com/kkoutini/PaSST/releases/download/v.0.0.6/esc50-passt-s-n-f128-p16-s10-fold2-acc.977.pt')
teacher.net.load_state_dict(state_dict)



 Loading PASST TRAINED ON AUDISET 




No pretrained weights exist or were found for this model. Using random initialization.


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (

No pretrained weights exist or were found for this model. Using random initialization.


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (

NameError: name 'nn' is not defined

In [11]:
out = teacher.net(torch.rand(1,1,128,256))

d:\tcc\Lung-Disease-Classification\models\passt.py:273: UserWarning: Input image size (128*256) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")


## Student Model

In [ ]:
student = make_model(net='xxs', num_classes = 4, patch_size=(4,4), patch_stride=(3,3))

## Models Hyperparams

In [ ]:
# Learning Rate
Lr = 0.001
# Optimizer
teacher_optim = AdamW(teacher.parameters(), lr=Lr)
student_optim = AdamW(student.parameters(), lr=Lr)

# Dataset

In [ ]:
train_df = pd.read_csv('train.cvs')
val_df   = pd.read_csv('Val.cvs')
test_df  = pd.read_csv('test.cvs')

In [ ]:
train_ds = train_set(train_df, 'Train')
val_ds = train_set(val_df, 'Val')
test_ds = train_set(test_df, 'Test')

In [ ]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)

val_loader = torch.utils.data.DataLoader(val_ds, batch_size=32, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_ds, batch_size=32, shuffle=True)

# Knwoledge Distillation 

In [ ]:
num_epoch=100
# Distillation instance
distiller = VanillaKD(teacher, student, train_loader, test_loader, teacher_optim, student_optim) 
# Train Teacher 
distiller.train_teacher(epochs=num_epoch, plot_losses=True, save_model=True)    
# Train Student
distiller.train_student(epochs=num_epoch, plot_losses=True, save_model=True)    